In [ ]:
import pandas as pd
import numpy as np
from operator import itemgetter
from itertools import permutations
import networkx as nx
import matplotlib.pyplot as plt
import csv

# Functions


In [ ]:
def load_ALL_TRADE_data():
  ### This is a function that loads accordingly the dataset from MACycle organization, which holds an Improved version of the UN Comtrade bulk.
  ### Returns a dataframe with clean data.
  data = pd.read_csv('trade_data_full.csv')
  data = data.drop(data[data['period'] == 201].index)     #deletion of false data
  data = data.drop(data[data['period'] == 20].index)
  data = data.drop(data[data['partnercode'].isna()].index)     #deletion of na data
  data = data.drop(data[data['new_tradevalue'].isna()].index)
  data = data[['period', 'reportercode', 'tradeflowcode', 'partnercode', 'new_tradevalue']]     #only conserving important columns.
  data['period'] = data['period'].astype('int16')     #casting from string to int identifiers.
  data['reportercode'] = data['reportercode'].astype('int16')
  data['tradeflowcode'] = data['tradeflowcode'].astype('int16')
  data['partnercode'] = data['partnercode'].astype('int16')
  data = data.replace({'reportercode':{490:158,251:250,699:356,381:380,579:578,757:756,842:840}})     #replacing country codes with new formal codes.
  data = data.replace({'partnercode':{490:158,251:250,699:356,381:380,579:578,757:756,842:840}})
  return data

def get_ISO_df():
    ### This is a function that obtains all official ISO codes for the final selected countries, numerical (ISO-3) and the original strings.
    ### Return a dataframe with ISO codes information.
    ISO_codes = pd.read_csv('FINAL ALL countries ISO.csv', encoding='latin1')     #obtain final country's list
    ISO_codes = ISO_codes[1:177]
    info = pd.read_csv('ISO_codes.csv')     #obtain ISO codes for each country
    info.columns = info.columns.str.replace('English short name lower case', 'Name')
    drop_c = ['Alpha-2 code', 'Name', 'ISO 3166-2']
    info.drop(drop_c, inplace=True, axis=1)
    dict_letters = {}
    for i in range(len(info)):
        dict_letters[info['Numeric code'].iloc[i]] = info['Alpha-3 code'].iloc[i]
    new_letters = []
    vals = list(ISO_codes['List of Countries ISO'].values)
    for i in range(len(ISO_codes)):
        if vals[i] == '531': new_letters.append('CUW')     #solve few discrepancies with countries ISO codes.
        elif vals[i] == '534': new_letters.append('SXM')
        else: new_letters.append(dict_letters.get(int(vals[i]), 0))
    ISO_codes['ISO-3'] = new_letters
    return ISO_codes

def convert_name_links(links):
    ### This is a function that converts the countries' name accordingly to the new formal form.
    ### Takes a dictionary of links.
    ### Returns THE SAME links with different country names.
    change = [['United States', 'United States Of America', 840],
         ['Korea, Rep. of', 'Korea, Republic of (South Korea)', 410],
         ['Russian Federation', 'Russia', 643],
         ['Taiwan Province of China', 'Taiwan', 158],
         ['Poland, Rep. of', 'Poland', 616],
         ['British Virgin Islands', 'Virgin Islands, British', 92],
         ['Curaçao, Kingdom of the Netherlands', 'Curaçao', 531],
         ['Czech Rep.', 'Czech Republic', 203],
         ['Kazakhstan, Rep. of', 'Kazakhstan', 398],
         ['Bahrain, Kingdom of', 'Bahrain', 48],
         ['Curaçao and Sint Maarten', 'Sint Maarten', 534],
         ['Venezuela, Rep. Bolivariana de', 'Venezuela', 862],
         ['Slovak Rep.', 'Slovakia', 703],
         ['Slovenia, Rep. of', 'Slovenia', 705],
         ['Bahamas, The', 'Bahamas', 44],
         ['Egypt, Arab Rep. of', 'Egypt', 818],
         ['Marshall Islands, Rep. of the', 'Marshall Islands', 584],
         ['Estonia, Rep. of', 'Estonia', 233],
         ['Croatia, Rep. of', 'Croatia', 191],
         ['Serbia, Rep. of', 'Serbia', 688],
         ['Dominican Rep.', 'Dominican Republic', 214],
         ['Aruba, Kingdom of the Netherlands', 'Aruba', 533],
         ['United States Virgin Islands', 'Virgin Islands, U.S.', 850],
         ['Kosovo, Rep. of', 'Kosovo', 999],
         ['Sint Maarten, Kingdom of the Netherlands', 'Sint Maarten', 534],
         ['Azerbaijan, Rep. of', 'Azerbaijan', 31],
         ['Belarus, Rep. of', 'Belarus', 112],
         ['Iran, Islamic Rep. of', 'Iran', 364],
         ['Congo, Dem. Rep. of the', 'Congo, the Democratic Republic of the', 180],
         ['Turks and Caicos Islands', 'Turks and Caicos', 796],
         ['North Macedonia, Republic of',
          'Macedonia, the former Yugoslav Republic of',
          807],
         ["Lao People's Dem. Rep.", "Lao People's Democratic Republic", 418],
         ['Tanzania, United Rep. of', 'Tanzania, United Republic of', 834],
         ['Armenia, Rep. of', 'Armenia', 51],
         ['Mozambique, Rep. of', 'Mozambique', 508],
         ['Andorra, Principality of', 'Andorra', 20],
         ['Ethiopia, The Federal Dem. Rep. of', 'Ethiopia', 231],
         ['Congo, Rep. of', 'Congo', 178],
         ['Mauritania, Islamic Rep. of', 'Mauritania', 478],
         ['Kyrgyz Rep.', 'Kyrgyzstan', 417],
         ['Syrian Arab Rep.', 'Syrian Arab Republic', 760],
         ['St. Kitts and Nevis', 'Saint Kitts and Nevis', 659],
         ['St. Vincent and the Grenadines', 'Saint Vincent and the Grenadines', 670],
         ['Bonaire, St. Eustatius and Saba', 'Bonaire, Sint Eustatius and Saba', 535],
         ['San Marino, Rep. of', 'San Marino', 674],
         ['Cabo Verde', 'Cape Verde', 132],
         ['Libya', 'Libyan Arab Jamahiriya', 434],
         ['Fiji, Rep. of', 'Fiji', 242],
         ['Eswatini, Kingdom of', 'Eswatini', 748],
         ['St. Lucia', 'Saint Lucia', 662],
         ['Palau, Rep. of', 'Palau', 585],
         ['Yemen, Rep. of', 'Yemen', 887],
         ['Madagascar, Rep. of', 'Madagascar', 450],
         ['Moldova, Rep. of', 'Moldova, Republic of', 498],
         ['Lesotho, Kingdom of', 'Lesotho', 426],
         ['Uzbekistan, Rep. of', 'Uzbekistan', 860],
         ['Tajikistan, Rep. of', 'Tajikistan', 762],
         ['Eritrea, The State of', 'Eritrea', 232],
         ['Guiana, French', 'French Guiana', 254],
         ['Afghanistan, Islamic Rep. of', 'Afghanistan', 4],
         ['Nauru, Rep. of', 'Nauru', 520],
         ['Central African Rep.', 'Central African Republic', 140],
         ['Equatorial Guinea, Rep. of', 'Equatorial Guinea', 226],
         ['Gambia, The', 'Gambia', 270],
         ['Saint Helena', 'Saint Helena, Ascension and Tristan da Cunha', 654],
         ['Micronesia, Federated States of', 'Micronesia, Federated States of', 583],
         ['Comoros, Union of the', 'Comoros', 174],
         ['Wallis and Futuna Islands', 'Wallis and Futuna', 876],
         ['Pitcairn Islands', 'Pitcairn', 612],
         ['Holy See', 'Holy See (Vatican City State)', 336],
                 ['China, P.R.: Hong Kong', 'Hong Kong', 344],
                 ['China, P.R.: Macao', 'Macao', 446],
                 ['China, P.R.: Mainland', 'China', 156],
                 ['Netherlands, The', 'Netherlands', 528],
             ['Türkiye, Rep of', 'Turkey', 792]]
    change_values = [k[0] for k in change]
    new_links = {}
    for i in list(links.keys()):
        (nfrom, nto) = i
        if nfrom in change_values:
            nfrom = change[change_values.index(nfrom)][1]
        if nto in change_values:
            nto = change[change_values.index(nto)][1]
        new_links[(nfrom,nto)] = links[i]
    return new_links

def removekey(d, key):
    r = dict(d)
    del r[key]
    return r

def get_ISO_codes_from_df(ISO_codes):
    dict_codes = {}
    for i in ISO_codes.values:
        dict_codes[i[0]] = i[1]
    return dict_codes

def ISO_dict_converter(links, ISO_codes):
    ISO_converter = get_ISO_codes_from_df(ISO_codes)
    final_data = {(ISO_converter[str(k[0])], ISO_converter[str(k[1])]): v for k, v in links.items()}
    return final_data

def get_reversedISO3_codes_from_df(ISO_codes):
    dict_codes = {}
    for i in list(ISO_codes.index):
        dict_codes[i] = ISO_codes.loc[i][1]
    return dict_codes

def ISO_dict_converter3(links, ISO_codes):
    ISO_converter = get_reversedISO3_codes_from_df(ISO_codes)
    final_data = {(ISO_converter[(k[0])], ISO_converter[(k[1])]): v for k, v in links.items()}
    return final_data

def get_trade_in_higher_Rank(a, b, rankA, rankB):
    return a if rankA < rankB else b

def perc_diff(a,b):
  ### This is a function that returns the percentage difference for 2 values.
  a = abs(a)
  b = abs(b)
  if a == 0 and b == 0:
    return 400
  elif a == 0 or b == 0:
    return 200
  elif a == b:
    return 0
  else:
    l = [a,b]
    l.sort(reverse=True)
    return (((l[0] - l[1]) / ((l[0] + l[1]) / 2)) * 100)

def get_bilateral_trade(data, year, ISO_codes):
  ### This is a function that obtains the trade flow for a desired year.
  ### Takes as input all trade data, a desired year, and the ISO codes information.
  ### Returns the desired year trade flow performing data curation as mentioned on the documentation.
  data = data[data['period'] == year]
  countries = list(get_ISO_codes_from_df(ISO_codes).keys())
  all_links = list(permutations(countries, 2))
  imports = {}
  exports = {}
  for i in range(len(data)):
      nfromi = data['reportercode'].iloc[i]
      ntoi = data['partnercode'].iloc[i]
      nfrom = nfromi
      nto = ntoi
      link = (nfrom, nto)
      flowcode = data['tradeflowcode'].iloc[i]
      value = data['new_tradevalue'].iloc[i]
      if (flowcode == 2) or (flowcode == 3):
        exports[link] = value + exports.get(link, 0)
      if (flowcode == 1) or (flowcode == 4):
        imports[link] = value + imports.get(link, 0)
  for key in imports:
    imports[key] = int(imports[key]/1000000)
  for key in exports:
    exports[key] = int(exports[key]/1000000)
  countries_diff = {}
  green_countries = {}
  for i in all_links:
    (cfromi, ctoi) = i
    (cfrom, cto) = (int(cfromi), int(ctoi))
    link = (cfrom, cto)
    a = exports.get(link,0)
    b = imports.get((cto,cfrom),0)
    diff = perc_diff(a,b)
    countries_diff[link] = diff
    if diff < 16:
        green_countries[cfrom] = green_countries.get(cfrom, 0) + 1
        green_countries[cto] = green_countries.get(cto, 0) + 1
  green_countries_rank = sorted(green_countries.items(), key=itemgetter(1), reverse = True)
  dgreen_countries_rank = {}
  for i in range(len(green_countries_rank)):
    dgreen_countries_rank[green_countries_rank[i][0]] = i + 1
  df_countries_diff = pd.DataFrame(list(countries_diff.items()),columns = ['Countries Link', 'Perc Diff'])
  df_countries_diff.sort_values(by=['Perc Diff'], inplace=True)
  final_value_links = {}
  for i in range(len(df_countries_diff)):
    link = df_countries_diff['Countries Link'].iloc[i]
    diff = df_countries_diff['Perc Diff'].iloc[i]
    (cfrom, cto) = link
    valueA = exports.get(link,0)
    valueB = imports.get((cto,cfrom),0)
    if diff < 16:
        final_value_links[link] = (valueA + valueB) / 2
    elif diff == 200:
        final_value_links[link] = max(valueA, valueB)
    else:
        rankA = dgreen_countries_rank.get(cfrom, 0)
        rankB = dgreen_countries_rank.get(cto, 0)
        final_value_links[link] = get_trade_in_higher_Rank(valueA, valueB, rankA, rankB)
  final_value_links = ISO_dict_converter(final_value_links, ISO_codes)
  return exports,imports, df_countries_diff, green_countries, final_value_links

def flatten(item):
    (i, j) = item[0]
    k = item[1]
    return [i,j,k]

def save_all_flow(flow, typeflow, year):
    ### This is a function that saves as a .csv all flow for a year
    with open('All ' + typeflow + ' Links ('+ str(year) + ').csv', 'w', newline='') as f:
        write = csv.writer(f)
        write.writerow(['All ' + typeflow + '  Links ('+ str(year) + ')'])
        write.writerow(['Country', 'Partner', 'Flow'])
        write.writerows(flow)

def save_all_flows(ISO_codes):
    ### This is a function that saves all years flows in different .csv
    TRADE_data = load_ALL_TRADE_data()
    allyears = list(range(2001, 2020))
    all_years_trade = []
    for year in allyears:
        all_years_trade.append(get_bilateral_trade(TRADE_data, year, ISO_codes))
    for year in range(0, 19):
        t = all_years_trade[year][4]
        t = {k: v for k, v in t.items() if v > 0}
        t = [flatten(k) for k in list(t.items())]
        save_all_flow(t, 'Trade', allyears[year])


# Code

In [ ]:
### CAREEFUL: TAKES A LARGE AMOUNT OF TIME ≈ 10-12m
ISO_codes = get_ISO_df()
save_all_flows(ISO_codes)